In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, AveragePooling2D, Flatten, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import os

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
class Data:
    def __init__(self, train_dir, buffer_size, batch_size, validation_split):
        self.validation_split = 0.2
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.train_dir = pathlib.Path(train_dir)
        self.CLASS_NAME = np.array([item.name for item in self.train_dir.glob('*') if item.name != 'LICENSE.txt'])
        
    def spliting_train_test(self):
        full_dataset = tf.data.Dataset.list_files(str(self.train_dir/'*/*'))
        full_dataset_size = len(full_dataset)
        train_size = int((1-self.validation_split)*full_dataset_size)
        train_dataset = full_dataset.take(train_size)
        validation_dataset = full_dataset.skip(train_size)
        return train_dataset, validation_dataset
        #shayad ye returni bekhad
    
    def get_label(self, data):
        p = tf.strings.split(data, os.sep)
        p = p[-2] == self.CLASS_NAME
        return tf.math.argmax(p, output_type=tf.int32)
    
    @staticmethod
    def augmentation(image):
        img = tf.image.resize_with_crop_or_pad(image, 180,180)
        img = tf.image.random_crop(img, (150,150,3))
        img = tf.image.random_brightness(img, max_delta=0.5)
        return img
    
    @staticmethod
    def load_image(data):
        img = tf.io.read_file(data)
        img = tf.image.decode_image(img, 3, expand_animations=False)
        img = tf.cast(img, tf.float32)
        return img

    
    @staticmethod
    def normalize(image):
        return image/127. - 1
    
    @staticmethod
    def resize(image,height, width):
        image = tf.image.resize(image, (height, width),
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        #image = tf.image.resize_with_crop_or_pad(image, height, width)
        return image
    
    def load_imge_with_label(self, data):
        label = self.get_label(data)
        img = self.load_image(data)
        return img, label
    
    
    def load_train_datset(self, data):
        img, label = self.load_imge_with_label(data)
        img = self.augmentation(img)
        img = self.normalize(img)
        return img, label
    
    def load_test_dataset(self, data):
        img, label = self.load_imge_with_label(data)
        img = self.resize(img, 150,150)
        img = self.normalize(img)
        return img, label
    
    def creating_dataset(self):
        train_dataset, validation_dataset = self.spliting_train_test()
        train_ds = train_dataset.map(self.load_train_datset)
        test_ds = validation_dataset.map(self.load_test_dataset)
        train_ds = train_ds.shuffle(1024).batch(64)
        test_ds = test_ds.batch(64)
        return train_ds, test_ds

In [ ]:
directory = 'F:\\TF\\tf.data\\flower_photos\\'
data = Data(directory, 1024, 64, 0.2)
train_dataset, validation_dataset = data.creating_dataset()

In [ ]:
class IdentityBlock(Model):
    def __init__(self, filters):
        super(IdentityBlock, self).__init__()
        f1, f2 = filters
        self.conv1 = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.01))
        self.bn1 = BatchNormalization()

        self.conv2 =  Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.01))
        self.bn2 = BatchNormalization()
        
        self.conv3 = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.01))
        self.bn3 = BatchNormalization()

        self.act = layers.Activation('relu')
        self.add = layers.Add()

    def call(self, inp):
        x_skip = inp
        x = self.conv1(inp)
        x = self.bn1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.add([x_skip, x])
        x = self.act(x)
        return x

In [ ]:
class ConvolutionalBlock(Model):
    def __init__(self, s, filters):
        super(ConvolutionalBlock, self).__init__()
        f1, f2 = filters
        self.conv1 = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.01))                         
        self.bn1 = BatchNormalization()

        self.conv2 = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.01))
        self.bn2 = BatchNormalization()
        
        self.conv3 = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.01))
        self.bn3 = BatchNormalization()
        
        self.conv4 = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.01))
        self.bn4 = BatchNormalization()

        self.act = layers.Activation('relu')
        self.add = layers.Add()

    def call(self, inp):
        x_skip = inp
        x = self.conv1(inp)
        x = self.bn1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x_skip = self.conv4(x_skip)
        x_skip = self.bn4(x_skip)
        x = self.add([x_skip, x])
        x = self.act(x)
        return x

In [ ]:
class ResNet(Model):
    def __init__(self, num_class):
        super(ResNet, self).__init__()
        self.conv = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))
        self.bn = BatchNormalization()
        self.act = layers.Activation('relu')
        self.max_pool = MaxPool2D((3, 3), strides=(2, 2))
        
        self.convb1 = ConvolutionalBlock(s=1, filters=(64, 256))
        self.idyb1 = IdentityBlock(filters=(64, 256))
        self.idyb2 = IdentityBlock(filters=(64, 256))
        
        self.convb2 = ConvolutionalBlock(s=2, filters=(128, 512))
        self.idyb3 = IdentityBlock(filters=(128, 512))
        self.idyb4 = IdentityBlock(filters=(128, 512))
        self.idyb5 = IdentityBlock(filters=(128, 512))
        
        self.convb3 = ConvolutionalBlock(s=2, filters=(256, 1024))
        self.idyb6 = IdentityBlock(filters=(256, 1024))
        self.idyb7 = IdentityBlock(filters=(256, 1024))
        self.idyb8 = IdentityBlock(filters=(256, 1024))
        self.idyb9 = IdentityBlock(filters=(256, 1024))
        self.idyb10 = IdentityBlock(filters=(256, 1024))
        
        self.convb4 = ConvolutionalBlock(s=2, filters=(512, 2048))
        self.idyb11 = IdentityBlock(filters=(512, 2048))
        self.idyb12 = IdentityBlock(filters=(512, 2048))
        
        self.ave_pool = AveragePooling2D((2, 2), padding='same')
        self.classifier = Dense(num_class, activation='softmax', kernel_initializer='he_normal')

    def call(self, inp):
        x = ZeroPadding2D(padding=(3, 3))(inp)
        x = self.conv(inp)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)
        
        x = self.convb1(x)
        x = self.idyb1(x)
        x = self.idyb2(x)
        
        x = self.convb2(x)
        x = self.idyb3(x)
        x = self.idyb4(x)
        x = self.idyb5(x)
        
        x = self.convb3(x)
        x = self.idyb6(x)
        x = self.idyb7(x)
        x = self.idyb8(x)
        x = self.idyb9(x)
        x = self.idyb10(x)
        
        x = self.convb4(x)
        x = self.idyb11(x)
        x = self.idyb12(x)
        
        x = self.ave_pool(x)
        x = Flatten()(x)
        x = self.classifier(x)
        return x

In [ ]:
resnet = ResNet(5)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = resnet.fit(train_dataset, validation_data=validation_dataset, epochs=50)